# Анализ базы данных сервиса для чтения книг по подписке

**Задача:** проанализировать переданную базу данных сервиса.

**Этапы:**
1. Определить, сколько книг вышло после 1 января 2000 года.
2. Для каждой книги посчитать количество обзоров и среднюю оценку.
3. Определить издательство, которое выпустило наибольшее число книг.
4. Определить автора с самой высокой средней оценкой книг (учитываем книги с 50 и более оценками).
5. Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Подготовка к работе и знакомство с данными

In [2]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

In [3]:
# устанавливаем параметры
db_config = {'user': ***, 
'pwd': ***, 
'host': ***,
'port': ***, 
'db': ***} 
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [4]:
# функция для выполнения sql-запроса
def select(query):
    return pd.io.sql.read_sql(query, con = engine)

In [5]:
# запрос для вывода строк таблицы books
books_query = '''
SELECT *
FROM books
LIMIT 5;
'''

# запрос для вывода строк таблицы authors
authors_query = '''
SELECT *
FROM authors
LIMIT 5;
'''

# запрос для вывода строк таблицы publishers
publishers_query = '''
SELECT *
FROM publishers
LIMIT 5;
'''

# запрос для вывода строк таблицы ratings
ratings_query = '''
SELECT *
FROM ratings
LIMIT 5;
'''

# запрос для вывода строк таблицы reviews
reviews_query = '''
SELECT *
FROM reviews
LIMIT 5;
'''

In [6]:
# выводим таблицы для ознакомления
display(select(books_query))
display(select(authors_query))
display(select(publishers_query))
display(select(ratings_query))
display(select(reviews_query))

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Количество книг, вышедших после 1 января 2000 года

In [7]:
count_books = '''
SELECT COUNT(DISTINCT book_id)
FROM books
WHERE publication_date >= '2000-01-02';
'''

select(count_books)

,count
0,819


In [8]:
all_books = '''
SELECT COUNT(DISTINCT book_id)
FROM books;
'''

select(all_books)

,count
0,1000


Всего с 1 января 2000 года в базу сервиса занесено 819 книг. В целом в базе 1000 наименований.

##  Количество обзоров и средняя оценка для каждой книги

In [9]:
reviews_books = '''
SELECT books.book_id, title, COUNT(DISTINCT review_id) AS reviews, AVG(rating) AS avg_rating
FROM books
INNER JOIN ratings ON books.book_id=ratings.book_id
INNER JOIN reviews ON books.book_id=reviews.book_id
GROUP BY books.book_id, title
ORDER BY reviews DESC;
'''

select(reviews_books)

,book_id,title,reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,854,The Road,6,3.772727
2,656,The Book Thief,6,4.264151
3,734,The Glass Castle,6,4.206897
4,963,Water for Elephants,6,3.977273
...,...,...,...,...
989,465,Naked Empire (Sword of Truth #8),1,3.500000
990,446,Moo Baa La La La!,1,3.000000
991,431,Merrick (The Vampire Chronicles #7),1,4.000000
992,92,Babyville,1,3.500000


У 6 книг отсутствуют отзывы и оценки.

Больше всего отзывов у первой книги серии Twilight. Большее число отзывов может снижать рейтинг книги, но дает более развернутое представление, тогда как максимальный рейтинг книг со всего одним отзывом нельзя считать реалистичным.

## Издательство, выпустившее наибольшее число книг

Мы учитываем только издания толще 50 страниц, чтобы исключить из анализа брошюры.

In [10]:
active_publisher = '''
SELECT p.publisher AS publisher_name, COUNT(DISTINCT b.book_id) AS books_count
FROM books AS b
INNER JOIN publishers AS p ON b.publisher_id=p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id
ORDER BY books_count DESC
LIMIT 1;
'''

select(active_publisher)

,publisher_name,books_count
0,Penguin Books,42


Наибольшее число книг выпустило издательство Penguin Books.

## Автор с самой высокой средней оценкой книг

Учитываем книги с числом оценок от 50 и более.

In [14]:
best_author = '''
SELECT b.book_id, b.author_id, COUNT(r.rating) AS sum_of_ratings, AVG(r.rating) AS avg_rating
FROM books AS b
INNER JOIN ratings AS r ON b.book_id=r.book_id
GROUP BY b.book_id
HAVING COUNT(r.rating) >= 50
'''
select(best_author)

,book_id,author_id,sum_of_ratings,avg_rating
0,75,106,84,3.678571
1,750,240,88,4.125000
2,545,630,66,3.787879
3,948,554,160,3.662500
4,488,311,61,3.622951
5,696,106,59,3.830508
6,722,240,74,4.391892
7,627,469,57,3.789474
8,733,372,56,3.750000
9,779,498,62,4.080645


In [17]:
best_author = '''
SELECT a.author, AVG(rb.avg_rating)
FROM (
SELECT b.book_id, b.author_id, COUNT(r.rating) AS sum_of_ratings, AVG(r.rating) AS avg_rating
FROM books AS b
INNER JOIN ratings AS r ON b.book_id=r.book_id
GROUP BY b.book_id
HAVING COUNT(r.rating) >= 50
) AS rb
INNER JOIN authors AS a ON rb.author_id=a.author_id
GROUP BY a.author
ORDER BY avg DESC
'''
select(best_author)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Выше всех оценены книги Джоан Роулинг.

## Cреднее количество обзоров от пользователей

Учитываем пользователей, которые оставили более 50 обзоров.

In [ ]:
active_users = '''
SELECT ROUND(AVG(ur.count)) AS avg_ratings
FROM (SELECT u.username, COUNT(review_id)
FROM reviews AS r
INNER JOIN (
SELECT username
FROM ratings
GROUP BY username
HAVING COUNT(rating) > 50
) AS u ON r.username=u.username
GROUP BY u.username) AS ur
'''

select(active_users)

Среднее число обзоров — 24.

## Выводы

Всего с 1 января 2000 года в базу сервиса занесено 819 книг. В целом в базе 1000 наименований.

Больше всего отзывов у первой книги серии Twilight. Большее число отзывов может снижать рейтинг книги, но дает более развернутое представление, тогда как максимальный рейтинг книг со всего одним отзывом нельзя считать реалистичным.

Наибольшее число книг выпустило издательство Penguin Books.

Выше всех оценены книги Джоан Роулинг.

Среднее число обзоров — 24.